In [3]:
pip install opencv-python

   ---------------------------------------- 0.0/39.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/39.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/39.0 MB ? eta -:--:--
   ---------------------------------------- 0.3/39.0 MB ? eta -:--:--
   ---------------------------------------- 0.3/39.0 MB ? eta -:--:--
   ---------------------------------------- 0.3/39.0 MB ? eta -:--:--
   ---------------------------------------- 0.3/39.0 MB ? eta -:--:--
    --------------------------------------- 0.5/39.0 MB 329.3 kB/s eta 0:01:57
    --------------------------------------- 0.5/39.0 MB 329.3 kB/s eta 0:01:57
    --------------------------------------- 0.5/39.0 MB 329.3 kB/s eta 0:01:57
    --------------------------------------- 0.8/39.0 MB 335.7 kB/s eta 0:01:54
    --------------------------------------- 0.8/39.0 MB 335.7 kB/s eta 0:01:54
    --------------------------------------- 0.8/39.0 MB 335.7 kB/s eta 0:01:54
    --------------------------------

In [4]:
import cv2
print(cv2.__version__)  # should print version number if installed

4.12.0


In [2]:
import cv2
import numpy as np
import os

# ✅ Path to dataset
input_folder = r"E:\A\OneDrive\Documents\3rd year\5th SEM Materials of SR.Mishra(3rd yr)\Deep Learning for Image Analytics\Project\Groundnut_Leaf_dataset\train"
output_folder = os.path.join(os.path.dirname(input_folder), "single_leaf_train")
os.makedirs(output_folder, exist_ok=True)

for class_folder in os.listdir(input_folder):
    class_path = os.path.join(input_folder, class_folder)
    if not os.path.isdir(class_path):
        continue

    save_class_path = os.path.join(output_folder, class_folder)
    os.makedirs(save_class_path, exist_ok=True)

    for filename in os.listdir(class_path):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            img_path = os.path.join(class_path, filename)
            image = cv2.imread(img_path)

            if image is None:
                print(f"⚠️ Could not read {img_path}")
                continue

            hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
            lower_green = np.array([25, 40, 40])
            upper_green = np.array([85, 255, 255])
            mask = cv2.inRange(hsv, lower_green, upper_green)

            contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            if not contours:
                print(f"⚠️ No leaf detected in {filename}")
                continue

            # Find largest contour (main leaf)
            largest_contour = max(contours, key=cv2.contourArea)

            # Create mask for largest contour only
            main_mask = np.zeros_like(mask)
            cv2.drawContours(main_mask, [largest_contour], -1, 255, thickness=cv2.FILLED)

            # Keep only main leaf
            leaf_only = cv2.bitwise_and(image, image, mask=main_mask)

            # Black background
            bg_black = np.zeros_like(image)
            final = np.where(main_mask[:, :, None] == 255, leaf_only, bg_black)

            # Crop tightly around main leaf
            x, y, w, h = cv2.boundingRect(largest_contour)
            cropped_final = final[y:y+h, x:x+w]

            save_path = os.path.join(save_class_path, filename)
            cv2.imwrite(save_path, cropped_final)

    print(f"✅ Processed class: {class_folder}")

print("\n🎯 Single leaf isolation completed.")
print(f"📂 Saved in: {output_folder}")

✅ Processed class: early_leaf_spot_1
✅ Processed class: early_rust_1
✅ Processed class: healthy_leaf_1
✅ Processed class: late_leaf_spot_1
✅ Processed class: nutrition_deficiency_1
✅ Processed class: rust_1

🎯 Single leaf isolation completed.
📂 Saved in: E:\A\OneDrive\Documents\3rd year\5th SEM Materials of SR.Mishra(3rd yr)\Deep Learning for Image Analytics\Project\Groundnut_Leaf_dataset\single_leaf_train
